In [ ]:
# This file computes LyC escape fractions 
# of the MagE spectra according to
# Rivera-Thorsen+19 (arXiv:1904.08186)

In [ ]:
import os
import glob

import numpy as np
import pandas as pd

from scipy.interpolate import interp1d

from astropy.io import fits

import stsynphot as STS

import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText

In [ ]:
home = os.getcwd()
data = home + '/data'
figs = home +'/figs'

#lycFiles = glob.glob(data + '/spectra/*.txt')

# Model spectrum
s99 = data + '/spectra/plankarc_reddy_ext.txt'

# Masks of the arc and MagE slits
arcMaskFile = data + '/hst/masks/arcMask.fits'
slitMaskFiles = [data + '/hst/masks/M5mask.fits',
    data + '/hst/masks/M4mask.fits',
    data + '/hst/masks/M6mask.fits',
    data + '/hst/masks/M3mask.fits',
    data + '/hst/masks/M0mask.fits',
    data + '/hst/masks/M2mask.fits',
    data + '/hst/masks/M7mask.fits',
    data + '/hst/masks/M8mask.fits',
    data + '/hst/masks/M9mask.fits'
    ]

# HST imaging in F275W and F814W filters
f275file = data + '/hst/V4.0_PSZ1G311.65-18.48_F275W_0.03g0.8_cr2.5_0.7_drc_sci.fits'
f814file = data + '/hst/V4.0_PSZ1G311.65-18.48_F814W_0.03g0.8_cr2.5_0.7_drc_sci.fits'

In [ ]:
df = pd.read_csv(data + '/spectra/mage/slits.txt', delimiter=' ', header=0)

slits = np.array(['M5', 'M4', 'M6', 'M3', 'M0', 'M2', 'M7', 'M8', 'M9'])

# String array containing MAGE spectra file names
mageFilesBase = df.iloc[:,1].to_numpy(dtype=str)[2:]
mageFiles = np.empty(np.shape(mageFilesBase), dtype=object)

for i, file in enumerate(mageFilesBase):
    file = data + '/spectra/mage/' + file
    mageFiles[i] = file
    
# Array containing redshifts of each slit
z = df.iloc[:,2].to_numpy(dtype=np.float64)[2:]

In [ ]:
df = pd.read_csv(s99, delimiter='\s+', header=None, engine='python')

w99 = df.iloc[:,0].to_numpy()
f99 = df.iloc[:,1].to_numpy()
ne99 = df.iloc[:,2].to_numpy()

In [ ]:
# IGM transmission according to Rivera-Thorsen+19
tIGM = 0.69

In [ ]:
arcMask = fits.getdata(arcMaskFile)

# Retrieve HST imaging data
f275 = fits.getdata(f275file)
f814 = fits.getdata(f814file)

f275 = np.where(np.isnan(f275), 0.0, f275)
f814 = np.where(np.isnan(f814), 0.0, f814)

f275 = np.where(f275 < 0.0, 0.0, f275)
f814 = np.where(f814 < 0.0, 0.0, f814)

# Convert HST imaging data to erg/s/cm^2/Å
f275 = f275 * fits.open(f275file)[0].header['PHOTFLAM']
f814 = f814 * fits.open(f814file)[0].header['PHOTFLAM']

bp275 = STS.band('wfc3,uvis1,f275w,mjd#58216')
w275 = bp275.binset
tpt275 = bp275(w275)
w275 = w275.value

bp814 = STS.band('acs,wfc1,f814w,mjd#58170')
w814 = bp814.binset
tpt814 = bp814(w814)
w814 = w814.value

int1 = np.trapz(tpt275[(w275 >= 2100.0) & (w275 <= 3250.0)], x=w275[(w275 >= 2100.0) & (w275 <= 3250.0)])
int2 = np.trapz(tpt814[(w814 >= 6500.0) & (w814 <= 10000.0)], x=w814[(w814 >= 6500.0) & (w814 <= 10000.0)])

# For each slit
for i, slit in enumerate(slits):
    
    #Convert .txt file with MagE data into a data frame
    df = pd.read_csv(mageFiles[i], delimiter='\t', header=0, skiprows=13)
    
    #Discard missing data
    df = df.apply(pd.to_numeric, errors='coerce').dropna()

    #Discard extreme outliers
    df = df[df.iloc[:,1] < 1e-20]

    #Reset the index
    df = df.reset_index(drop=True)

    # Extract wavelength, flux, and noise
    w = df.iloc[:,0].to_numpy()
    f = df.iloc[:,1].to_numpy() * 2.998e18 / np.square(w)
    ne = df.iloc[:,2].to_numpy() * 2.998e18 / np.square(w)
    
    # Place data into rest frame of the source
    w = w / (1.0 + z[i])
    f = f * (1.0 + z[i])
    ne = ne * (1.0 + z[i])
    
    # Interpolate MagE flux density at 1500 Å
    interpMage = interp1d(w, f)
    f1500Mage = interpMage(1500.0)
    
    # Interpolate S99 flux density at 1500 Å
    interpS99 = interp1d(w99, f99)
    f1500S99 = interpS99(1500.0)
    
    # Scale S99 to match MagE 1500 Å flux density
    f99 = f99 * (f1500Mage / f1500S99)
    ne99 = ne99 * (f1500Mage / f1500S99)
    
    w99m = w99 * (1.0 + z[i])
    f99m = f99 / (1.0 + z[i])
    
    #print(np.shape(w275), np.shape(tpt275))
    
    interpT275 = interp1d(w275, tpt275)
    tpt275interp = interpT275(w99m[(w99m >= 2100.0) & (w99m <= 3250.0)])
    
    interpT814 = interp1d(w814, tpt814)
    tpt814interp = interpT814(w99m[(w99m >= 6500.0) & (w99m <= 10000.0)])
    
    int3 = np.trapz(tpt275interp * f99m[(w99m >= 2100.0) & (w99m <= 3250.0)], x=w99m[(w99m >= 2100.0) & (w99m <= 3250.0)])
    int4 = np.trapz(tpt814interp * f99m[(w99m >= 6500.0) & (w99m <= 10000.0)], x=w99m[(w99m >= 6500.0) & (w99m <= 10000.0)])
    
    slitMask = fits.getdata(slitMaskFiles[i])
    
    # Intersection between slit and arc masks
    intr = arcMask * slitMask
    
    f275flux = np.sum(intr * f275)
    f814flux = np.sum(intr * f814)
    
    #print()
    
    #print(f275flux, f814flux, int1, int2, int3, int4)
    
    esc = (f275flux * int1 * int4) / (tIGM * f814flux * int2 * int3)
    
    print(round(esc.value * 100.0, 1))